# Comparison of different Classification Algorithms



In [6]:
import sklearn
import numpy as np
import pandas as pd

from main import aggregate_data, normalize_data
from metrics.githubMetrics import GithubMetrics, metricCollection
from importer.testDataImporter import TestDataImporter

In [13]:
importer = TestDataImporter('data/testset.csv')
y_train = np.array(importer.trainset.classification)
y_test = np.array(importer.testset.classification)

In [10]:
data_train = aggregate_data(importer.trainset.repos)
data_train = normalize_data(data_train)
data_train[:5]

,avg_entropy,avg_folder_depth,repo_size,up_to_dateness,watcher_count,file_count,forks_count,file_folder_ratio,open_issue_count
0,0.251590,0.253733,0.752917,0.026821,0.770764,0.405086,0.566949,0.330773,0.641663
1,0.270911,0.333086,0.370083,0.708447,0.753491,0.136859,0.534790,0.084179,0.151630
2,0.431880,0.236519,0.735055,0.597203,0.000000,0.484467,0.000000,0.275341,0.000000
3,0.232237,0.736634,0.373685,0.498483,0.000000,0.344582,0.000000,0.169776,0.000000
4,0.259969,0.534729,0.508795,0.873993,0.000000,0.349585,0.000000,0.210574,0.000000


In [12]:
data_test = aggregate_data(importer.testset.repos)
data_test = normalize_data(data_test)
data_test[:5]

,avg_entropy,avg_folder_depth,repo_size,up_to_dateness,watcher_count,file_count,forks_count,file_folder_ratio,open_issue_count
0,0.564294,0.310194,0.747260,0.539362,0.421838,0.252792,0.730426,0.191077,0.150190
1,0.741162,0.339471,0.723308,0.877016,0.000000,0.197237,0.000000,0.020757,0.000000
2,0.769716,0.359101,0.408197,0.971735,0.073646,0.167862,0.000000,0.014088,0.000000
3,0.527382,0.282112,0.763555,0.766765,0.073646,0.123329,0.485355,0.048594,0.834246
4,0.516772,0.804519,0.724402,0.201886,0.441875,0.538326,0.427203,0.162831,0.555771


## Logistic Regression

In [14]:
logreg = sklearn.linear_model.LogisticRegression(C=1.0, max_iter=100, n_jobs=2)
logreg.fit(data_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=2,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
logreg.coef_

array([[ 0.22457466, -1.19206779, -0.26185993, -0.64466193,  1.40746356,
        -0.90217353,  0.43948792,  0.5656693 , -0.82029758],
       [-1.02194912,  0.37297587,  1.21066623, -1.3657322 , -0.88031193,
         1.39492158, -1.34278266,  0.89368225,  0.96252042],
       [ 1.30662512, -1.55369588,  0.37402214, -0.27016482,  0.04955791,
        -0.33191972, -0.58647011, -0.52736235,  0.15644162],
       [-0.23836844, -0.24501599, -0.24225152, -0.19036835,  1.08265998,
        -0.61391072,  1.73642814, -0.52485589, -0.70562363],
       [-0.65939469, -0.32206543, -0.36361769,  0.18770769, -1.04133017,
         0.67372841,  0.2266435 , -0.24770463, -0.39197227],
       [-0.44397304,  0.55295501, -3.15828118,  0.53443706, -0.73306548,
        -1.95426283, -0.75385378, -1.31927554,  0.00546681],
       [ 0.21905238, -0.84946616,  0.0931602 , -1.59805084, -0.99551816,
        -0.32892676, -0.70549327, -0.19199932, -0.48216327]])

In [22]:
print('acc train:', logreg.score(data_train, y_train))
print('acc test:', logreg.score(data_test, y_test))

acc train: 0.477663230241
acc test: 0.462025316456


'l2'